In [5]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup

# Step 1: Create a session and load the page
def years(start_year, end_year):
    urls = []
    for year in range(start_year,end_year +1):
        url = 'https://www.nba.com/stats/teams/drives?Season='+str(year-1)+'-'+str((year)%2000)+'&PerMode=PerGame'
        urls.append(url)
    return urls
url_list = years(2014,2023)
def get_tables(url_list):
    data = []
    for url in url_list:
        print(url)
        driver = webdriver.Chrome()
        driver.get(url)

        # Wait for the page to fully load
        driver.implicitly_wait(8)

        # Step 2: Parse HTML code and grab tables with Beautiful Soup
        soup = BeautifulSoup(driver.page_source, 'lxml')

        tables = soup.find_all('table')

        # Step 3: Read tables with Pandas read_html()
        dfs = pd.read_html(str(tables))

        #print(f'Total tables: {len(dfs)}')
        #print(dfs[2].head())
    
        driver.close()
        #return dfs
        df= dfs[-1]
        #drop = ['Unnamed: 16_level_1', 'Unnamed: 17_level_1', 'Unnamed: 18_level_1']
        #df.columns = df.columns.droplevel()
        #df = df.drop(columns = drop)
        data.append(df)
    return data
frames = get_tables(url_list)

https://www.nba.com/stats/teams/drives?Season=2013-14&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2014-15&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2015-16&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2016-17&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2017-18&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2018-19&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2019-20&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2020-21&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2021-22&PerMode=PerGame
https://www.nba.com/stats/teams/drives?Season=2022-23&PerMode=PerGame


In [9]:
year = 2014
for df in frames:
    df['year'] = year
    df.to_csv(str(year)+'drives.csv',index = False)
    year+=1
    

In [12]:
drives = pd.concat(frames)

In [29]:
drives = drives[drives.year != 2020]

In [26]:
test = get_tables(['https://www.nba.com/stats/teams/drives?Season=2021-22&PerMode=PerGame'])

https://www.nba.com/stats/teams/drives?Season=2021-22&PerMode=PerGame


In [36]:
test['year'] = 2020

In [39]:
drives = drives.dropna(subset = ['year'])
drives

,Team,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,...,PF,PF%,year,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
0,Atlanta Hawks,81.0,38.0,43.0,241.9,38.6,6.2,15.0,41.5,3.7,...,2.4,6.1,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Boston Celtics,82.0,25.0,57.0,240.6,30.4,5.0,12.0,41.9,3.4,...,2.3,7.5,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brooklyn Nets,81.0,43.0,38.0,242.5,36.5,6.4,14.4,44.2,5.1,...,3.4,9.3,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Charlotte Bobcats,81.0,42.0,39.0,242.8,34.9,5.8,14.0,41.3,4.8,...,3.1,8.9,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chicago Bulls,82.0,48.0,34.0,243.1,26.8,4.1,10.7,38.7,3.0,...,2.0,7.5,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Sacramento Kings,14.0,8.0,6.0,241.8,50.4,10.6,18.6,56.7,8.1,...,5.1,10.2,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,San Antonio Spurs,16.0,6.0,10.0,241.6,49.0,9.9,22.3,44.5,4.8,...,3.0,6.1,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Toronto Raptors,16.0,9.0,7.0,240.0,42.3,8.3,18.7,44.1,5.4,...,3.3,7.7,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Utah Jazz,17.0,11.0,6.0,243.0,53.9,10.5,22.5,46.9,6.2,...,3.9,7.2,2023.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
drives = pd.concat([drives,test])
drives

,Team,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,...,PF,PF%,year,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
0,Atlanta Hawks,81.0,38.0,43.0,241.9,38.6,6.2,15.0,41.5,3.7,...,2.4,6.1,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Boston Celtics,82.0,25.0,57.0,240.6,30.4,5.0,12.0,41.9,3.4,...,2.3,7.5,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brooklyn Nets,81.0,43.0,38.0,242.5,36.5,6.4,14.4,44.2,5.1,...,3.4,9.3,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Charlotte Bobcats,81.0,42.0,39.0,242.8,34.9,5.8,14.0,41.3,4.8,...,3.1,8.9,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chicago Bulls,82.0,48.0,34.0,243.1,26.8,4.1,10.7,38.7,3.0,...,2.0,7.5,2014.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Sacramento Kings,81.0,29.0,52.0,241.5,51.1,10.6,22.3,47.7,5.7,...,3.8,7.4,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,San Antonio Spurs,82.0,34.0,48.0,241.5,52.7,11.2,23.6,47.6,5.3,...,3.4,6.4,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Toronto Raptors,82.0,48.0,34.0,242.1,44.1,8.5,18.9,44.9,4.2,...,2.7,6.2,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Utah Jazz,82.0,49.0,33.0,240.6,51.9,11.7,22.8,51.3,5.3,...,3.2,6.2,2020.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
drives['year'] = drives['year'].astype(int)

In [42]:
drives.to_csv('drives.csv',index = False)

In [43]:
df = pd.read_csv('drives.csv')

In [61]:
drives

,Team,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,...,PF,PF%,year,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
0,Atlanta Hawks,81.0,38.0,43.0,241.9,38.6,6.2,15.0,41.5,3.7,...,2.4,6.1,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Boston Celtics,82.0,25.0,57.0,240.6,30.4,5.0,12.0,41.9,3.4,...,2.3,7.5,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Brooklyn Nets,81.0,43.0,38.0,242.5,36.5,6.4,14.4,44.2,5.1,...,3.4,9.3,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Charlotte Bobcats,81.0,42.0,39.0,242.8,34.9,5.8,14.0,41.3,4.8,...,3.1,8.9,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Chicago Bulls,82.0,48.0,34.0,243.1,26.8,4.1,10.7,38.7,3.0,...,2.0,7.5,2014,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25,Sacramento Kings,81.0,29.0,52.0,241.5,51.1,10.6,22.3,47.7,5.7,...,3.8,7.4,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
26,San Antonio Spurs,82.0,34.0,48.0,241.5,52.7,11.2,23.6,47.6,5.3,...,3.4,6.4,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
27,Toronto Raptors,82.0,48.0,34.0,242.1,44.1,8.5,18.9,44.9,4.2,...,2.7,6.2,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
28,Utah Jazz,82.0,49.0,33.0,240.6,51.9,11.7,22.8,51.3,5.3,...,3.2,6.2,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
by_year = drives.groupby('year').max()[['DRIVES','FTA','PF','PTS%','Team']]

In [65]:
by_year['ftperdrive'] = by_year['FTA']/by_year['DRIVES']

In [66]:
by_year

,DRIVES,FTA,PF,PTS%,Team,ftperdrive
year,,,,,,
2014,42.6,6.9,3.5,59.1,Washington Wizards,0.161972
2015,47.4,7.1,3.7,56.0,Washington Wizards,0.149789
2016,50.7,7.5,3.8,59.2,Washington Wizards,0.147929
2017,49.4,7.6,3.9,58.0,Washington Wizards,0.153846
2018,52.9,6.6,3.3,59.5,Washington Wizards,0.124764
2019,55.1,8.6,4.2,61.3,Washington Wizards,0.156080
2020,61.7,8.3,4.2,64.9,Washington Wizards,0.134522
2021,61.7,8.2,4.1,66.5,Washington Wizards,0.132901
2022,61.7,8.3,4.2,64.9,Washington Wizards,0.134522


In [83]:
drives.groupby(["year","Team"]).max()["DRIVES"]

year  Team              
2014  Atlanta Hawks         38.6
      Boston Celtics        30.4
      Brooklyn Nets         36.5
      Charlotte Bobcats     34.9
      Chicago Bulls         26.8
                            ... 
2023  Sacramento Kings      50.4
      San Antonio Spurs     49.0
      Toronto Raptors       42.3
      Utah Jazz             53.9
      Washington Wizards    38.7
Name: DRIVES, Length: 300, dtype: float64

In [86]:
drives.groupby(['year','DRIVES']).head(1)
drives.columns

Index(['Team', 'GP', 'W', 'L', 'MIN', 'DRIVES', 'FGM', 'FGA', 'FG%', 'FTM',
       'FTA', 'FT%', 'PTS', 'PTS%', 'PASS', 'PASS%', 'AST', 'AST%', 'TO',
       'TOV%', 'PF', 'PF%', 'year', 'Sun', 'Mon', 'Tues', 'Wed', 'Thurs',
       'Fri', 'Sat'],
      dtype='object')

In [85]:
drives.sort_values(by ='DRIVES',ascending = False)

,Team,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,...,PF,PF%,year,Sun,Mon,Tues,Wed,Thurs,Fri,Sat
20,Oklahoma City Thunder,16.0,7.0,9.0,244.7,65.9,13.9,28.3,49.3,5.8,...,3.6,5.5,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Oklahoma City Thunder,82.0,24.0,58.0,241.5,61.7,11.4,25.2,45.2,6.4,...,4.2,6.8,2022,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Oklahoma City Thunder,82.0,24.0,58.0,241.5,61.7,11.4,25.2,45.2,6.4,...,4.2,6.8,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,Oklahoma City Thunder,72.0,22.0,50.0,241.0,61.7,11.5,25.1,45.7,6.1,...,4.1,6.6,2021,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,New York Knicks,16.0,8.0,8.0,243.1,55.8,13.3,27.1,49.0,6.4,...,3.9,6.9,2023,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,Minnesota Timberwolves,81.0,16.0,65.0,241.5,26.5,4.8,11.6,41.8,3.6,...,2.3,8.8,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Golden State Warriors,82.0,73.0,9.0,242.4,26.3,5.9,12.1,48.6,3.2,...,2.1,8.0,2016,NaN,NaN,NaN,NaN,NaN,NaN,NaN
19,New York Knicks,81.0,17.0,64.0,242.2,24.6,4.2,10.0,42.1,3.4,...,2.2,8.8,2015,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,Golden State Warriors,82.0,58.0,24.0,240.6,23.9,5.0,10.4,47.9,3.0,...,1.8,7.7,2018,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [88]:
drives = drives.drop(columns = ['Sun', 'Mon', 'Tues', 'Wed', 'Thurs','Fri', 'Sat'])

In [91]:
drives = drives.sort_values(by = 'DRIVES',ascending = False)

In [93]:
drives.to_csv('drives.csv',index = False)

In [116]:
drives.iloc[drives.reset_index().groupby(['year'])['DRIVES'].idxmax()]

,Team,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,...,PTS%,PASS,PASS%,AST,AST%,TO,TOV%,PF,PF%,year
22,Philadelphia 76ers,82.0,19.0,63.0,242.1,42.6,8.2,20.0,41.1,4.9,...,51.5,11.7,27.5,3.1,7.2,3.2,7.6,3.5,8.3,2014
22,Philadelphia 76ers,82.0,18.0,64.0,241.5,47.4,7.4,18.4,40.1,4.6,...,41.9,16.4,34.6,3.9,8.2,4.1,8.7,3.7,7.8,2015
22,Philadelphia 76ers,82.0,10.0,72.0,241.5,50.7,7.5,17.9,42.0,4.1,...,38.6,19.5,38.4,5.4,10.6,4.1,8.0,3.0,6.0,2016
15,Miami Heat,82.0,41.0,41.0,241.2,49.4,8.3,18.9,44.0,3.7,...,42.6,18.4,37.3,5.1,10.3,3.5,7.0,2.6,5.3,2017
28,Utah Jazz,82.0,48.0,34.0,240.9,52.9,9.2,20.4,45.2,4.3,...,44.1,18.0,34.1,6.0,11.3,4.0,7.5,2.7,5.0,2018
28,Utah Jazz,81.0,50.0,31.0,240.9,55.1,10.5,22.0,47.8,5.1,...,48.0,23.9,43.3,6.3,11.4,4.4,8.0,3.2,5.8,2019
20,Oklahoma City Thunder,82.0,24.0,58.0,241.5,61.7,11.4,25.2,45.2,6.4,...,48.5,26.3,42.7,5.1,8.3,4.1,6.7,4.2,6.8,2020
20,Oklahoma City Thunder,72.0,22.0,50.0,241.0,61.7,11.5,25.1,45.7,6.1,...,48.4,26.1,42.3,5.0,8.1,4.6,7.5,4.1,6.6,2021
20,Oklahoma City Thunder,82.0,24.0,58.0,241.5,61.7,11.4,25.2,45.2,6.4,...,48.5,26.3,42.7,5.1,8.3,4.1,6.7,4.2,6.8,2022
20,Oklahoma City Thunder,16.0,7.0,9.0,244.7,65.9,13.9,28.3,49.3,5.8,...,52.4,28.1,42.7,6.9,10.4,3.9,5.9,3.6,5.5,2023


In [112]:
drives.groupby(['year'])['DRIVES'].idxmax(axis = 0)

year
2014    22
2015    22
2016    22
2017    15
2018    28
2019    28
2020    20
2021    20
2022    20
2023    20
Name: DRIVES, dtype: int64

In [121]:
drives[drives.year == 2014]

,Team,GP,W,L,MIN,DRIVES,FGM,FGA,FG%,FTM,...,PTS%,PASS,PASS%,AST,AST%,TO,TOV%,PF,PF%,year
22,Philadelphia 76ers,82.0,19.0,63.0,242.1,42.6,8.2,20.0,41.1,4.9,...,51.5,11.7,27.5,3.1,7.2,3.2,7.6,3.5,8.3,2014
18,New Orleans Pelicans,82.0,34.0,48.0,242.1,42.0,8.6,20.1,42.5,4.7,...,53.3,11.0,26.2,3.9,9.4,2.9,6.9,3.3,7.8,2014
10,Houston Rockets,82.0,54.0,28.0,241.8,41.9,7.5,16.6,44.9,5.4,...,49.7,13.9,33.2,3.9,9.3,3.0,7.3,3.5,8.3,2014
7,Denver Nuggets,82.0,36.0,46.0,240.9,39.1,6.6,15.3,42.8,4.5,...,46.0,13.6,34.6,3.8,9.8,2.4,6.1,3.1,7.9,2014
15,Miami Heat,82.0,54.0,28.0,242.4,38.9,7.3,14.2,51.5,4.0,...,49.5,14.0,35.9,3.9,10.1,2.9,7.6,2.8,7.2,2014
0,Atlanta Hawks,81.0,38.0,43.0,241.9,38.6,6.2,15.0,41.5,3.7,...,42.6,14.1,36.4,3.8,9.8,2.8,7.1,2.4,6.1,2014
27,Toronto Raptors,81.0,47.0,34.0,243.4,38.1,6.4,15.0,43.0,4.4,...,46.4,13.0,33.9,3.9,10.2,2.6,6.9,2.8,7.2,2014
5,Cleveland Cavaliers,82.0,33.0,49.0,243.1,37.5,7.2,16.1,44.7,3.4,...,49.0,12.0,32.1,3.4,9.1,2.8,7.6,2.3,6.2,2014
2,Brooklyn Nets,81.0,43.0,38.0,242.5,36.5,6.4,14.4,44.2,5.1,...,49.9,11.7,31.9,3.3,9.1,2.8,7.7,3.4,9.3,2014
23,Phoenix Suns,82.0,48.0,34.0,240.9,36.1,6.7,14.7,45.7,3.8,...,48.9,12.3,34.1,3.3,9.2,2.6,7.1,2.6,7.2,2014
